# Inverted Bell-Curve based Ensemble of Deep Learning Models for Detection of COVID-19 from Chest X-rays

Novel Coronavirus 2019 disease or COVID-19 is a viral disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). Use of Chest X-Rays (CXRs) has become an important practice to assist in the diagnosis of COVID-19 as they can be used to detect the abnormalities developed in the infected patients' lungs. With the fast spread of the disease, many researchers across the world are striving to use several deep learning based systems to identify the COVID-19 from such CXR images. To this end, we propose an Inverted Bell-curve based ensemble of deep learning models for detection of COVID-19 from CXR images. We first use a selection of models pretrained on ImageNet dataset and use the concept of transfer learning to retrain them with CXR datasets. Then the trained models are combined with the proposed Inverted Bell Curve weighted ensemble method, where the output of each classifier is assigned a weight, and the final prediction is done by performing weighted average of those outputs. We evaluate the proposed method on two publicly available datasets: the COVID-19 Radiography Database and the IEEE COVID Chest X-Ray Dataset. The accuracy, F1 score and the AUC ROC achieved by the proposed method are 99.66\%, 99.75\% and 99.99\% respectively in the first dataset, and 99.84\%, 99.81\%, 99.99\% respectively in the other dataset. Experimental results ensure that the use of transfer learning based models and their combination using the proposed ensemble method result in improved predictions of COVID-19 in CXRs.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils import data
import numpy as np
import torchvision
from  numpy import exp,absolute
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math


path = '/content/drive/My Drive/COVIDxA/train/'

plt.ion()   
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


data_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(path,transform=data_transforms)

val_split = 0.8
train_size = math.floor(len(dataset)*val_split)
val_size = len(dataset) - train_size
trainset, valset = data.random_split(dataset,lengths=[train_size,val_size])
dataset_sizes = {'train':train_size,'val':val_size}

dataloaders = {
    'train': data.DataLoader(trainset,batch_size=16,shuffle=True),
    'val' : data.DataLoader(valset,batch_size=16,shuffle=True)
}

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

inputs, classes = next(iter(dataloaders['train']))
class_names = dataset.classes
num_classes = len(class_names)
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])



def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                print('bruh')

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

model_vgg = torchvision.models.vgg16(pretrained=True)
model_resnet = torchvision.models.resnet18(pretrained=True)
model_alexnet = torchvision.models.alexnet(pretrained=True)
model_dnet = torchvision.models.densenet161(pretrained=True)
for param in model_alexnet.parameters():
    param.requires_grad = True
for param in model_vgg.parameters():
    param.requires_grad = True
for param in model_resnet.parameters():
    param.requires_grad = True
for param in model_dnet.parameters():
    param.requires_grad = True
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, num_classes)
model_vgg.classifier[6] = nn.Linear(4096,num_classes)
model_alexnet.classifier[6] = nn.Linear(4096,num_classes)
model_dnet.classifier = nn.Linear(2208,num_classes)

model_vgg = model_vgg.to(device)
model_resnet = model_resnet.to(device)
model_alexnet = model_alexnet.to(device)
model_dnet = model_dnet.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_resnet.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=70, gamma=0.1)
model_resnet = train_model(model_resnet, criterion, optimizer_conv, exp_lr_scheduler,num_epochs=30)
torch.save(model_resnet, '/content/drive/My Drive/kaggle_resnet.pth')

optimizer_conv = optim.SGD(model_vgg.parameters(), lr=0.001, momentum=0.9)
model_vgg = train_model(model_vgg, criterion, optimizer_conv, exp_lr_scheduler,num_epochs=20)
torch.save(model_vgg, '/content/drive/My Drive/kaggle_vgg.pth')


optimizer_conv = optim.SGD(model_dnet.parameters(), lr=0.001, momentum=0.9)
model_dnet = train_model(model_dnet, criterion, optimizer_conv, exp_lr_scheduler,num_epochs=20)
torch.save(model_dnet, '/content/drive/My Drive/kaggle_dense.pth')

Let there be $C$ classes in dataset and $k$ classifiers trained on the dataset. In this paper value of $k$ is taken is 3 but $k$ can take any finite value. Let $s_i^j$ be the confidence score for $j^{th}$ class predicted by the $i^{th}$ classifier. The confidence scores are the output of softmax, hence output of some $i^{th}$ classifier will follow:
\begin{equation}
    \sum_{j=0}^C s_i^j = 1 \quad\quad {where}\quad s_i^j\space \in \space[0,1]
\end{equation}

Now weight is assigned to each of the classifiers output using inverted Bell curve function which is a function in form of
\begin{equation}
    f(x) = \frac{1}{a} exp(\frac{(x-b)^2}{2c^2})
\end{equation}

The function $f(x)$ is also known as inverted Bell curve. The inverted bell-shape is particularly useful to implement this weighted averaging scheme. It can be observe that shape of $f(x)$ is more round at the bottom than any equivalent parabolic curve. We hypothesise that this helps in penalising a wider range of low confidence score values, resulting in a better ensemble.

The parameter $a$ is inversly proportional to the depth of the inverted bell. The value of $a$ gets closer to $0$ the bottom of the curve comes nearer to the $x$-axis. The parameter $b$ controls the position of the centre of the curve bottom. At $x=b$, we can achieve the minimum value of $f(x)$ where $a > 0$. The parameter $c$ determines the width of the bell.

<p align="center">
<img src="https://user-images.githubusercontent.com/31564734/121768236-daaab000-cb7a-11eb-9a29-07dc69c9c56f.jpg" width="500px"/>
</p>

Let us consider the point $x=b$, where $f(x)$ has its minima given $a > 0$, so as $x$ is incremented or decremented we will get higher values of $f(x)$, similar amount at both direction due to the fact that $(x-b)$ term is squared in the equation. This very idea is used in the context of assigning weights to the outputs of each classifiers.

Let us consider two independent classifiers $P$ and $Q$ produce $[0.8,0.1,0.1]$ and $[0.5,0.3,0.2]$ as output confidence scores for some input $X$. Though, both of these classifiers predict the $X$ belongs to class-0, the classifier $P$ does it more confidently. Therefore, while doing weighted average of these scores, we must assign more weight to the classifier $P$ for this output. In doing so, the property of $f(x)$ discussed above is used. Let the minima of $f(x)$ is at $x=0.5$, then we will get higher values of $f(x)$ as we get closer to 0 or 1 because these are respectively lower and upper bounds for $s_i^j$. It can be easily shown that minima of $f(x)$ exists at $x=b$. So the value of $b$ is taken as 0.5 to satisfy our requirement. The value of $c$ determines the range of the weights and it is chosen as 0.5 experimentally .


There may arrive a situation when some classifiers having very poor performance metrics over a dataset but for some instances it produces the outputs confidently. Therefore, without suppressing the classifiers' impacts completely, we aim to weaken its contribution in the ensemble output, and we consider the accuracy of the classifier by taking $a=1/A_i.$ in $f(x)$, where $A_i$ is the accuracy for the $i^{th}$ classifier. So the wight $w_i$ assigned to the output of $i^{th}$ classifier is 

\begin{equation}
    w_i = A_i\cdot\sum_{j=0}^C f(s_i^j) \quad where
\end{equation}

\begin{equation}
    f(x) = exp(\frac{(x - 0.5)^2}{0.5})
\end{equation}


The final output $[Y_0, Y_1, ..., Y_C]$ is generated by taking the weighted average of confidence scores across $k$ classifiers using $w_i$, where
\begin{equation}
    Y_j = \frac{1}{k}\cdot \sum_{i=0}^k w_i\cdot s_i^j
\end{equation}

We can further apply softmax on the calculated $Y$ to normalise the output scores and obtain the final class probability. Finally the class $\zeta$ is predicted from this output as
\begin{equation}
    \zeta = {arg max}_{j} \{Y_j\}
\end{equation}

In [ ]:
def ensemble(op_classifiers):
    cf = 0.0
    for c in op_classifiers:
        c = c.cpu().numpy()
        w = np.identity(c.shape[0])
        i = 0
        for x in c:
            w[i][i] = np.sum(exp((x-0.5)*(x-0.5)/0.5))
            i += 1
        cf += np.matmul(w,c)
    cf = (cf)/len(op_classifiers)
    return torch.from_numpy(cf)

soft = nn.Softmax(dim=1)
running_corrects = 0

for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.set_grad_enabled(False):
        outputs_resnet = soft(model_resnet(inputs))
        outputs_vgg = soft(model_vgg(inputs))
        outputs_dnet = soft(model_dnet(inputs))
        outputs_alexnet = soft(model_alexnet(inputs))
        outputs = ensemble({outputs_dnet,outputs_resnet,outputs_vgg})
        _, preds = torch.max(outputs, 1)
        
        running_corrects += torch.sum(preds == labels.data)
print(running_corrects/dataset_sizes['val'])